In [ ]:
import sqlite3

#create or connect to database file
conn = sqlite3.connect("Deadlock.db")
cursor = conn.cursor()

#test
cursor.execute("SELECT sqlite_version();")
print("SQLite Version:", cursor.fetchone()[0])

In [ ]:
cursor.execute("""
CREATE TABLE hero_traits (
    hero_id INTEGER,
    trait_id INTEGER,
    trait_value INTEGER CHECK(trait_value BETWEEN 0 AND 100),
    PRIMARY KEY (hero_id, trait_id),
    FOREIGN KEY (hero_id) REFERENCES heroes(hero_id),
    FOREIGN KEY (trait_id) REFERENCES traits(trait_id)
    );
""")
conn.commit()

In [ ]:
# Insert heroes
cursor.executemany("""
INSERT INTO heroes (name, role) VALUES (?, ?)
""", [
    ("Volt", "Assassin"),
    ("Gravitas", "Tank"),
    ("Echo", "Support")
])

# Insert traits
cursor.executemany("""
INSERT INTO traits (trait_name, description) VALUES (?, ?)
""", [
    ("Mobility", "Speed and movement ability"),
    ("Sustain", "Healing or damage resistance"),
    ("Burst", "Short-term high damage"),
    ("Crowd Control", "Disables, slows, etc."),
    ("Range", "Attack or ability distance")
])

# Insert trait values
cursor.executemany("""
INSERT INTO hero_traits (hero_id, trait_id, trait_value) VALUES (?, ?, ?)
""", [
    (1, 1, 90),  # Volt - Mobility
    (1, 3, 85),  # Volt - Burst
    (2, 2, 80),  # Gravitas - Sustain
    (2, 4, 70),  # Gravitas - Crowd Control
    (3, 2, 75),  # Echo - Sustain
    (3, 5, 60)   # Echo - Range
])

conn.commit()


In [ ]:
for row in cursor.execute("""
SELECT h.name, t.trait_name, ht.trait_value
FROM hero_traits ht
JOIN heroes h ON ht.hero_id = h.hero_id
JOIN traits t ON ht.trait_id = t.trait_id
ORDER BY h.name;
"""):
    print(row)

In [ ]:
cursor.execute("""
CREATE TABLE patch_notes(
    patch_id INTEGER PRIMARY KEY AUTOINCREMENT,
    patch_date TEXT,
    description TEXT
);
""")
    

In [ ]:
cursor.execute("""
CREATE TABLE patch_changes(
    patch_id INTEGER,
    hero_id INTEGER,
    trait_id INTEGER,
    change_type TEXT CHECK (change_type IN ('buff', 'nerf', 'rework')),
    magintude INTEGER
    raw_note_text TEXT,
    FOREIGN KEY (patch_id) REFERENCES patch_notes(patch_id),
    FOREIGN KEY (hero_id) REFERENCES heroes(hero_id),
    FOREIGN KEY (trait_id) REFERENCES traits(trait_id)
);
""")

In [ ]:
conn.commit()

In [ ]:
cursor.execute("""
ALTER TABLE patch_changes RENAME COLUMN magintude TO magnitude
""")

In [ ]:
conn.commit()

In [ ]:
# Insert trait changes from that patch
cursor.executemany("""
INSERT INTO patch_changes (patch_id, hero_id, trait_id, change_type, magnitude, raw_note_text)
VALUES (?, ?, ?, ?, ?, ?)
""", [
    (1, 1, 3, 'nerf', -10, 'Volt’s burst damage reduced by 10%'),
    (1, 3, 2, 'buff', 15, 'Echo’s sustain healing increased by 15%')
])


In [ ]:
for row in cursor.execute("SELECT * FROM patch_changes"):
    print(row)

In [ ]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM patch_changes", conn)
df

In [ ]:
cursor.execute("""
ALTER TABLE patch_changes ADD COLUMN raw_note_text TEXT
""")
conn.commit()

In [ ]:
# Insert a sample patch
cursor.execute("""
INSERT INTO patch_notes (patch_date, description)
VALUES ('2025-03-01', 'Patch 1.1: Balance updates for Volt and Echo')
""")

# Insert trait changes from that patch
cursor.executemany("""
INSERT INTO patch_changes (patch_id, hero_id, trait_id, change_type, magnitude, raw_note_text)
VALUES (?, ?, ?, ?, ?, ?)
""", [
    (1, 1, 3, 'nerf', -10, 'Volt’s burst damage reduced by 10%'),
    (1, 3, 2, 'buff', 15, 'Echo’s sustain healing increased by 15%')
])

conn.commit()

In [ ]:
cursor.execute("""
SELECT h.name, t.trait_name, pc.change_type, pc.magnitude, pc.raw_note_text
FROM patch_changes pc
JOIN heroes h ON pc.hero_id = h.hero_id
JOIN traits t ON pc.trait_id = t.trait_id
WHERE pc.patch_id = 1
""")

for row in cursor.fetchall():
    print(row)

In [ ]:
cursor.execute("""
SELECT patch_id, patch_date, description
FROM patch_notes
ORDER BY patch_date DESC
""")

for row in cursor.fetchall():
    print(row)

In [ ]:
cursor.execute("SELECT * FROM patch_notes")
for row in cursor.fetchall():
    print(row)

In [ ]:
cursor.execute("SELECT patch_id FROM patch_notes")
for row in cursor.fetchall():
    print(row)

In [ ]:
cursor.execute("DELETE FROM patch_notes WHERE patch_id = 2")
cursor.execute("DELETE FROM patch_changes WHERE patch_id = 2")
cursor.execute("DELETE FROM patch_notes WHERE patch_id = 1")
cursor.execute("DELETE FROM patch_changes WHERE patch_id = 1")

conn.commit()

In [ ]:
import pandas as pd
df = pd.read_parquet("/Users/MWOfficeDesktop/OneDrive/Documents/Code/Deadlock Database/heroes.parquet")

# Display all columns
pd.set_option('display.max_columns', None)
df.info()

In [ ]:
pip install pyarrow

In [1]:
import sqlite3

#create or connect to database file
conn = sqlite3.connect("Deadlock.db")
cursor = conn.cursor()

#test
cursor.execute("SELECT sqlite_version();")
print("SQLite Version:", cursor.fetchone()[0])

SQLite Version: 3.45.3


In [2]:
cursor.execute("""
CREATE TABLE matches(
    start_time INTEGER,
    match_id INTEGER,
    game_mode INTEGER,
    PRIMARY KEY (match_id)
    
);
""")

In [4]:
cursor.execute("""
CREATE TABLE match_player(
    match_id INTEGER,
    account_id INTEGER,
    hero_id INTEGER,
    PRIMARY KEY (match_id, account_id),
    FOREIGN  KEY (account_id) REFERENCES account(account_id)
    FOREIGN  KEY (hero_id) REFERENCES heroes(heroes_id)
    
);
""")